# CAD Laboratory (CE4P001) - Assignment No. 2


**Name:** Amiya Das  <br>
**Roll No.:** 22CE01057 <br>

<h3>Question-4</h3>

Find out the stress distribution by modeling the given rectangular plate of 2 mm thick with a circular hole. The dimensions of the steel plate and hole are shown in Figure 1. The section is made up of steel having a value of **210 GPa** for Young's modulus and a value of **0.3** for Poisson's ratio.

![image.png](attachment:c375cf8e-3893-4e73-ad1a-8b3d06eabb55.png)

<div style="margin-left: 40px">

(a) Provide the steps for finite element meshing both in Julia and Abaqus. <br>
(b) Mention the governing PDEs that are required to solve for the stress distribution. <br>
(c) Provide a comparison between the results obtained from Gridap in Julia and Abaqus.

</div>

In [1]:
import Gmsh: gmsh

# input parameters
L = 1000;           # Length
H = 400;            # Breadth
h = 30;             # Height

# Initializing Gmsh (Initialize the Gmsh API)
gmsh.initialize()

# Create a Model (For example, let's create a rectangle of length L and height H)
gmsh.model.add("Rectangle")

# Define points
p1 = gmsh.model.geo.addPoint(0, 0, 0, h)
p2 = gmsh.model.geo.addPoint(L, 0, 0, h)
p3 = gmsh.model.geo.addPoint(L, H, 0, h)
p4 = gmsh.model.geo.addPoint(0, H, 0, h)

# Add lines
l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p4)
l4 = gmsh.model.geo.addLine(p4, p1)

# Create a rectangular curve loop and surface
c0 = gmsh.model.geo.addCurveLoop([l1, l2, l3, l4])

# Create circle arcs
p5 = gmsh.model.geo.addPoint(L/2, (H/2)-75, 0, h)
p6 = gmsh.model.geo.addPoint(L/2, H/2, 0, h)
p7 = gmsh.model.geo.addPoint(L/2, (H/2)+75, 0, h)

c1 = gmsh.model.geo.addCircleArc(p5, p6, p7)
c2 = gmsh.model.geo.addCircleArc(p7, p6, p5)

# Create circular curve loops
circle = gmsh.model.geo.addCurveLoop([c1,c2])

# Create plane surface
surface = gmsh.model.geo.addPlaneSurface([c0, -circle])

# Create physical group for domain
gmsh.model.addPhysicalGroup(2, [surface], 10, "Domain")

# Create physical groups for boundaries
gmsh.model.addPhysicalGroup(1, [l1], 1, "Bottom")
gmsh.model.addPhysicalGroup(1, [l2], 2, "Right")
gmsh.model.addPhysicalGroup(1, [l3], 3, "Top")
gmsh.model.addPhysicalGroup(1, [l4], 4, "Left")

# Synchronize and Mesh (Synchronize the CAD model and generate a mesh:)
gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)

# Create the folder (if it doesn't exist)
write_dir = "./Model"
isdir(write_dir) || mkpath(write_dir)  # creates the folder if missing

# Write mesh to file
gmsh.write(joinpath(write_dir, "Plate_hole.msh"))

#Launch GUI or Finalize (Launch the GUI to visualize the mesh or finalize the Gmsh API:)
gmsh.fltk.run()  # Launch GUI

# or 
gmsh.finalize() # Finalize Gmsh API

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 40%] Meshing curve 3 (Line)
Info    : [ 60%] Meshing curve 4 (Line)
Info    : [ 70%] Meshing curve 5 (Circle)
Info    : [ 90%] Meshing curve 6 (Circle)
Info    : Done meshing 1D (Wall 0.00636292s, CPU 0s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0347841s, CPU 0.015625s)
Info    : 598 nodes 1201 elements
Info    : Writing './Model\Plate_hole.msh'...
Info    : Done writing './Model\Plate_hole.msh'
-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Windows64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netge

In [2]:
using Gridap
using GridapGmsh

In [3]:
# Creating Diectory
mesh_name = "Plate_hole.msh"
mesh_file = joinpath(@__DIR__, "Model", mesh_name)
isfile(mesh_file) || error("Path does not exist: $mesh_file")

true

In [4]:
model = GmshDiscreteModel(mesh_file)

Info    : Reading 'c:\Users\IIT BBSR\Desktop\Amiya\CAD_Lab\CAD_Lab_Assignments\A2\Question_4\Model\Plate_hole.msh'...
Info    : 14 entities
Info    : 597 nodes
Info    : 1178 elements
Info    : Done reading 'c:\Users\IIT BBSR\Desktop\Amiya\CAD_Lab\CAD_Lab_Assignments\A2\Question_4\Model\Plate_hole.msh'


UnstructuredDiscreteModel()

In [5]:
# create vtu file for checkup
result_path = joinpath(@__DIR__, "..", "..", "..", "..", "Model", "Q_4", "model_check")
isdir(result_path) || mkpath(result_path)
writevtk(model, joinpath(result_path, "model_check"));

In [6]:
const E = 210000.0                              # N/mm^2
const ν = 0.3                                   # Poissons Ratio

const μ = E/(2*(1+ν))
const λ1 = (E*ν)/((1+ν)*(1-2*ν))
const λ = (2 * λ1 * μ) / (λ1 + 2 * μ)          # Plane stress correction

g = VectorValue(100.0, 0.0)                    # Force

σ(ε) = λ*tr(ε)*one(ε) + 2*μ*ε

σ (generic function with 1 method)

In [ ]:
# Γ_fixed   = tags = "Left"
# Γ_Loaded = tags = "Right"

In [7]:
order = 1

reffe = ReferenceFE(lagrangian,VectorValue{2,Float64},order)
V0 = TestFESpace(model,reffe;
    conformity=:H1,
    dirichlet_tags=["Left"],
    dirichlet_masks=[(true,true)])
  
g1 = VectorValue(0.0,0.0)
U = TrialFESpace(V0,[g1])

TrialFESpace()

In [8]:
degree = 2*order
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)
Γ_load  = BoundaryTriangulation(model, tags = "Right")
dΓ_N = Measure(Γ_load,degree)

GenericMeasure()

In [9]:
# Weak from

# Internal work (Stiffness component)
a(u,v) = ∫( ε(v) ⊙ (σ∘ε(u)) )*dΩ 

# External Work
l(v) = ∫(v⋅g)*dΓ_N

l (generic function with 1 method)

In [10]:
# Solve
op = AffineFEOperator(a,l,U,V0)
uh = solve(op)

SingleFieldFEFunction():
 num_cells: 1082
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 8254605993968048815

In [11]:
# create vtu file for checkup
result_path = joinpath(@__DIR__, "..", "..", "..", "..", "Model", "Q_4", "Result")
isdir(result_path) || mkpath(result_path)

"c:\\Users\\IIT BBSR\\Desktop\\Amiya\\CAD_Lab\\CAD_Lab_Assignments\\A2\\Question_4\\..\\..\\..\\..\\Model\\Q_4\\Result"

In [12]:
writevtk(Ω,joinpath(result_path, "results_1"),
    cellfields=[
        "Displacement"=>uh,
        "Strain"=>ε(uh),
        "Stress"=>σ∘ε(uh)]
        )

(["c:\\Users\\IIT BBSR\\Desktop\\Amiya\\CAD_Lab\\CAD_Lab_Assignments\\A2\\Question_4\\..\\..\\..\\..\\Model\\Q_4\\Result\\results_1.vtu"],)